In [1]:
import requests
import json
from tqdm import tqdm

In [2]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    'currency':'KRW',
    'locale':'ko',
    'listing_id':'6261834',
    'role':'guest',
    '_format':'for_p3',
    '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
    '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
    '_order':'language_country'}

##### https://www.airbnb.co.kr/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=KRW&locale=ko&listing_id=6261834&role=guest&_format=for_p3&_limit=7&_offset=7&_order=language_country

In [3]:
response = requests.get(url,params=params).text

In [4]:
result = json.loads(response)

- result:{'reviews', 'metadata'}
    -  reviews:[100 items]: {'collection_tag', __'comments'__, 'created_at', __'id'__, 'language', 'localized_date', 'rating', 'response', 'reviewee', __'reviewer'__}
    - metadata:{'reviews_count', 'should_show_review_translations'}

In [5]:
for k,v in result['reviews'][0].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||                 collection_tag || : None
||                       comments || : 너무 깨끗하고 좋았습니다.
||                     created_at || : 2018-10-29T07:04:45Z
||                             id || : 342585456
||                       language || : ko
||                 localized_date || : 2018년 10월
||                         rating || : 5
||                       response || : 
||                       reviewee || : {'deleted': False, 'first_name': '현아', 'host_name': '현아', 'id': 24468677, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/5af5aaac-5719-4708-8056-b32d76f23b44.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/24468677'}
||                       reviewer || : {'deleted': False, 'first_name': 'Doohoon', 'host_name': 'Doohoon', 'id': 81515153, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/user/c9774fbd-6469-4db7-81a1-f374743fcad8.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/81515153'}


### remarketing_id를 이용한 리뷰데이터 크롤링

In [6]:
# 306개 숙소 정보담은 home_jsn_306,json 파일 불러오자
with open('home_jsn_306.json','r') as fp: 
    home_jsn = json.load(fp)

In [7]:
# remarketing_ids 라는 key로 숙소 id 리스트가 저장되어있다
remarketing_ids = home_jsn['explore_tabs'][0]['home_tab_metadata']['remarketing_ids']
len(remarketing_ids)

306

306개 숙소의 모든 리뷰를 긁어오자

In [8]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
# header에 User Agent를 붙여서 bot이 아님을 보여줌

params = {
        'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
        'currency':'KRW',
        'locale':'ko',
#         'listing_id':
        'role':'guest',
        '_format':'for_p3',
        '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
#         '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
        '_order':'language_country'
            }

review_jsn ={} # 리뷰데이터를 json으로 저장하기위해 우선 dict 생성

for home_id in tqdm(remarketing_ids): # 숙소id home_id에 대해
    review_jsn[home_id] = [] # home_id를 키로 갖는 리뷰리스트 쌓자
    
    params['listing_id'] = home_id # 리스팅 id를 해당 home_id로 설정
    
    retry_cnt = 2 # 최대 2번동안 home_id에 해당하는 response 요청하겠다 
    while retry_cnt !=0:
        try:
            home_id_res = requests.get(url, params=params, headers=headers).json()
            break
        except Exception as e: # response 못받은 경우 retry
            print('retry in home_id!!',e)
            retry_cnt -= 1
            time.sleep(2)
            continue
            
    review_count= home_id_res['metadata']['reviews_count']
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수
    
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
        params['_offset'] = 100*i
        retry_cnt = 2
        
        while retry_cnt !=0: # 최대 2번 동안 request
            try:
                res_100_reviews = requests.get(url, params=params, headers=headers).json()
                break
            except Exception as e: # response 못받은 경우 retry
                print('retry in review!!',e)
                retry_cnt -= 1
                time.sleep(2)
                continue
        
     
        review_jsn[home_id].extend(res_100_reviews['reviews']) # home_id 를 key로 100개의 리뷰데이터를 쌓음
        
            
    params['limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
    while retry_cnt !=0: # 최대 2번 동안 request
            try:
                res_rest_reviews = requests.get(url, params=params, headers=headers).json()
                break
            except Exception as e: # response 못받은 경우 retry
                print('retry in review!!',e)
                retry_cnt -= 1
                time.sleep(2)
                continue
                
    review_jsn[home_id].extend(res_rest_reviews['reviews'])

100%|██████████| 306/306 [19:10<00:00,  3.24s/it]


결과를 json 파일로 저장하자

In [9]:
with open('review_jsn_306.json','w') as fp:
    json.dump(review_jsn,fp,ensure_ascii=False)

불러오자

- review_dict: {home_id: [review_lst]} 형태이다

In [10]:
with open('review_jsn_306.json','r') as fp:
    review_dict = json.load(fp)

In [11]:
review_dict['8213750'][0]

{'collection_tag': None,
 'comments': '한국에서 비앤비를 이용해본건 처음이었는데 기대이상으로 너무 좋았어요 :D \n방은 사진이랑 거의 같았고 크리스마스 분위기에 맞게 설치된 조명과 인원에 맞게 준비해주신 룸슬리퍼, 마스크팩, 직접 손으로 쓴 카드까지!! 감동의 서비스였어요♡.♡\n여행객들뿐만 아니라 모임이나 파티할 장소가 필요하신 분들에게도 좋은 선택이 될 것 같아요. 서울역 바로앞이라 위치도 정말 좋고 아쉬운 점이 하나도 없는 숙소였네요 :)',
 'created_at': '2015-12-25T13:36:37Z',
 'id': 57403381,
 'language': 'ko',
 'localized_date': '2015년 12월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'Kenny & Kelly',
  'host_name': 'Kenny & Kelly',
  'id': 37875586,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/user/20a67e57-e49e-493a-965b-8c361e33984d.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/37875586'},
 'reviewer': {'deleted': False,
  'first_name': 'Mijin',
  'host_name': 'Mijin',
  'id': 21864948,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/users/21864948/profile_pic/1411899978/original.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/sh